In [2]:
# Import 
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.pandas.set_option("display.max_columns" , None)
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

In [3]:
test_df = pd.read_csv("/Users/krushna/Downloads/house-prices-advanced-regression-techniques-2/test.csv")

In [5]:
# Missing Values
# Number of features with missing values 
features_with_null_values_test =  [features for features in test_df.columns if test_df[features].isnull().sum() > 1]

#percentage of values in the missing features 
for feature in features_with_null_values_test:
    print(feature, "=",np.round(test_df[feature].isnull().mean() , 4), '%' )

MSZoning = 0.0027 %
LotFrontage = 0.1556 %
Alley = 0.9267 %
Utilities = 0.0014 %
MasVnrType = 0.6127 %
MasVnrArea = 0.0103 %
BsmtQual = 0.0302 %
BsmtCond = 0.0308 %
BsmtExposure = 0.0302 %
BsmtFinType1 = 0.0288 %
BsmtFinType2 = 0.0288 %
BsmtFullBath = 0.0014 %
BsmtHalfBath = 0.0014 %
Functional = 0.0014 %
FireplaceQu = 0.5003 %
GarageType = 0.0521 %
GarageYrBlt = 0.0535 %
GarageFinish = 0.0535 %
GarageQual = 0.0535 %
GarageCond = 0.0535 %
PoolQC = 0.9979 %
Fence = 0.8012 %
MiscFeature = 0.965 %


In [7]:
# Numerical Variables 
numerical_features_test = [features for features in test_df.columns if test_df[features].dtypes != 'O']
print("Numerical Features = " ,len(numerical_features_test))

test_df[numerical_features_test].head()

Numerical Features =  37


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1961.0,1.0,730.0,140,0,0,0,120,0,0,6,2010
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1958.0,1.0,312.0,393,36,0,0,0,0,12500,6,2010
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,1997.0,2.0,482.0,212,34,0,0,0,0,0,3,2010
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,324.0,926.0,926,678,0,1604,0.0,0.0,2,1,3,1,7,1,1998.0,2.0,470.0,360,36,0,0,0,0,0,6,2010
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,1017.0,1280.0,1280,0,0,1280,0.0,0.0,2,0,2,1,5,0,1992.0,2.0,506.0,0,82,0,0,144,0,0,1,2010


In [8]:
# Date Time Variables
year_features_test = [features for features in numerical_features_test if 'Yr' in features or 'Year'in features]
print(year_features_test,"\n")

for features in year_features_test:
    print(features, test_df[features].unique())
    print("\n")

['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold'] 

YearBuilt [1961 1958 1997 1998 1992 1993 1990 1970 1999 1971 1975 2009 2005 2003
 2002 2006 2004 1920 1974 2000 2010 1967 1978 1966 1964 1962 1952 1949
 1959 1956 1955 1948 1960 1963 1900 1957 1938 1923 1890 1910 1922 1950
 1925 1939 1940 1942 1936 1930 1915 1912 1917 1907 1968 1977 1927 1947
 1954 1987 1996 1972 1945 1924 1931 1941 1994 1989 1921 1896 1976 2008
 1965 1973 1984 1985 1991 1969 1995 1980 2007 1986 1981 1951 1953 1935
 1902 1946 1934 1983 1880 1979 2001 1916 1918 1988 1937 1905 1928 1926
 1929 1914 1895 1982 1879 1901 1919 1932]


YearRemodAdd [1961 1958 1998 1992 1994 2007 1990 1970 1999 1971 1975 2010 2005 2006
 2004 2002 1950 1993 2000 2003 1967 1978 1966 1964 2001 2008 2009 1959
 1956 1952 1955 1960 1984 1954 1957 1962 1997 1980 1968 1987 1979 1996
 1972 1985 1991 1989 1977 1976 1973 1986 1969 1995 1974 1981 1965 1951
 1953 1963 1982 1988 1983]


GarageYrBlt [1961. 1958. 1997. 1998. 1992. 1993. 1990. 1970. 1999.

In [9]:
# Numerical Feature 
# Two Types: Discrete and Continous 
# Lets check how discrete make a change to Sales Price

descrete_features_test = [features for features in numerical_features_test if len(test_df[features].unique())<25 and features not in year_features_test +['Id']]
print("Total descrete_features: " , len(descrete_features_test))

Total descrete_features:  16


In [10]:
# Continous Varibale 
continous_feature_test = [features for features in numerical_features_test if features not in descrete_features_test+ year_features_test+['id']]
print("Total continous_features: " , len(continous_feature_test))

Total continous_features:  17


In [11]:
# Categorical Features

categorical_features_test = [features for features in test_df.columns if test_df[features].dtypes == 'O']
test_df[categorical_features_test].head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RH,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,NaN,TA,TA,CBlock,TA,TA,No,Rec,LwQ,GasA,TA,Y,SBrkr,TA,Typ,NaN,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,TA,TA,CBlock,TA,TA,No,ALQ,Unf,GasA,TA,Y,SBrkr,Gd,Typ,NaN,Attchd,Unf,TA,TA,Y,NaN,NaN,Gar2,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,NaN,TA,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Gd,Y,SBrkr,TA,Typ,TA,Attchd,Fin,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,TA,TA,PConc,TA,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Gd,Attchd,Fin,TA,TA,Y,NaN,NaN,NaN,WD,Normal
4,RL,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,Gable,CompShg,HdBoard,HdBoard,NaN,Gd,TA,PConc,Gd,TA,No,ALQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,NaN,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [12]:
# Missing Values
# Number of features with missing values - Categorical features
categorical_features_with_null_values_test =  [features for features in test_df.columns if test_df[features].isnull().sum() > 1 and test_df[features].dtypes == 'O']
#percentage of values in the missing features 
for feature in features_with_null_values_test:
    print(feature, "=",np.round(test_df[feature].isnull().mean() , 4), '%' )

MSZoning = 0.0027 %
LotFrontage = 0.1556 %
Alley = 0.9267 %
Utilities = 0.0014 %
MasVnrType = 0.6127 %
MasVnrArea = 0.0103 %
BsmtQual = 0.0302 %
BsmtCond = 0.0308 %
BsmtExposure = 0.0302 %
BsmtFinType1 = 0.0288 %
BsmtFinType2 = 0.0288 %
BsmtFullBath = 0.0014 %
BsmtHalfBath = 0.0014 %
Functional = 0.0014 %
FireplaceQu = 0.5003 %
GarageType = 0.0521 %
GarageYrBlt = 0.0535 %
GarageFinish = 0.0535 %
GarageQual = 0.0535 %
GarageCond = 0.0535 %
PoolQC = 0.9979 %
Fence = 0.8012 %
MiscFeature = 0.965 %


In [13]:
# Replace missing values with a new label 

def replace_cat_features_test(data, name_features):
    data7 = data.copy()
    data7[name_features] = data7[name_features].fillna('Mising')
    return data7

test_df = replace_cat_features_test(test_df,categorical_features_with_null_values_test)
test_df[features_with_null_values_test].isnull().sum()

MSZoning          0
LotFrontage     227
Alley             0
Utilities         0
MasVnrType        0
MasVnrArea       15
BsmtQual          0
BsmtCond          0
BsmtExposure      0
BsmtFinType1      0
BsmtFinType2      0
BsmtFullBath      2
BsmtHalfBath      2
Functional        0
FireplaceQu       0
GarageType        0
GarageYrBlt      78
GarageFinish      0
GarageQual        0
GarageCond        0
PoolQC            0
Fence             0
MiscFeature       0
dtype: int64

In [14]:
# Missing Values
# Number of features with missing values - Numerical Featres
numerical_features_with_null_values_test =  [features for features in test_df.columns if test_df[features].isnull().sum() > 1 and test_df[features].dtypes != 'O']
#percentage of values in the missing features 
for feature in features_with_null_values_test:
    print(feature, "=",np.round(test_df[feature].isnull().mean() , 4), '%' )

MSZoning = 0.0 %
LotFrontage = 0.1556 %
Alley = 0.0 %
Utilities = 0.0 %
MasVnrType = 0.0 %
MasVnrArea = 0.0103 %
BsmtQual = 0.0 %
BsmtCond = 0.0 %
BsmtExposure = 0.0 %
BsmtFinType1 = 0.0 %
BsmtFinType2 = 0.0 %
BsmtFullBath = 0.0014 %
BsmtHalfBath = 0.0014 %
Functional = 0.0 %
FireplaceQu = 0.0 %
GarageType = 0.0 %
GarageYrBlt = 0.0535 %
GarageFinish = 0.0 %
GarageQual = 0.0 %
GarageCond = 0.0 %
PoolQC = 0.0 %
Fence = 0.0 %
MiscFeature = 0.0 %


In [15]:
# Replace missing values with a median value or 1 or 0

for feature in numerical_features_with_null_values_test:
    #calculate median
    median_c = test_df[feature].median()
    
    #create new feature to capture nan values
    # if nan- replace it with 1
    # else - 0 
    test_df[feature +'nan'] = np.where(test_df[feature].isnull(),1,0)
    test_df[feature].fillna(median_c, inplace=True)

test_df[numerical_features_with_null_values_test].isnull().sum()

LotFrontage     0
MasVnrArea      0
BsmtFullBath    0
BsmtHalfBath    0
GarageYrBlt     0
dtype: int64

In [16]:
# Now lets check with DateTime variable 

for feature in year_features_test:
    test_df[feature] = test_df['YrSold'] - test_df[feature]

In [17]:
## Log transformation for Numerical Features 
numerical_features_with_nonzero_values_test = [col for col in numerical_features_test if col != 'Id' and not (test_df[col] == 0).any()]
test_df[numerical_features_with_nonzero_values_test] = np.log(test_df[numerical_features_with_nonzero_values_test])
numerical_features_with_nonzero_values_test

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 '1stFlrSF',
 'GrLivArea',
 'TotRmsAbvGrd',
 'MoSold']

In [19]:
test_data = test_df.copy()
numerical_cols_test = test_df.select_dtypes(include=['int', 'float']).columns

In [21]:
scaler = MinMaxScaler()
test_df[numerical_cols_test] = scaler.fit_transform(test_df[numerical_cols_test])

In [23]:
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontagenan,MasVnrAreanan,BsmtFullBathnan,BsmtHalfBathnan,GarageYrBltnan
0,0.000000,0.000000,RH,0.593445,0.566360,Pave,Mising,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,0.698970,0.815465,0.384615,0.822581,Gable,CompShg,VinylSd,VinylSd,Mising,0.000000,TA,TA,CBlock,TA,TA,No,Rec,0.116708,LwQ,0.094364,0.126168,0.173111,GasA,TA,Y,SBrkr,0.312253,0.000000,0.0,0.312253,0.0,0.0,0.25,0.0,0.333333,0.5,TA,0.317394,Typ,0.00,Mising,Attchd,0.792994,Unf,0.2,0.490591,TA,TA,Y,0.098315,0.000000,0.0,0.0,0.208333,0.0,Mising,MnPrv,Mising,0.000000,0.721057,0.0,WD,Normal,0.0,0.0,0.0,0.0,0.0
1,0.000686,0.000000,RL,0.598957,0.622527,Pave,Mising,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,0.778151,0.815465,0.407692,0.870968,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,0.083721,TA,TA,CBlock,TA,TA,No,ALQ,0.230175,Unf,0.000000,0.189720,0.260844,GasA,TA,Y,SBrkr,0.468253,0.000000,0.0,0.468253,0.0,0.0,0.25,0.5,0.500000,0.5,Gd,0.430677,Typ,0.00,Mising,Attchd,0.802548,Unf,0.2,0.209677,TA,TA,Y,0.275983,0.048518,0.0,0.0,0.000000,0.0,Mising,Mising,Gar2,0.735294,0.721057,0.0,WD,Normal,0.0,0.0,0.0,0.0,0.0
2,0.001372,0.487992,RL,0.558854,0.614005,Pave,Mising,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,0.698970,0.732487,0.107692,0.225806,Gable,CompShg,VinylSd,VinylSd,Mising,0.000000,TA,TA,PConc,Gd,TA,No,GLQ,0.197257,Unf,0.000000,0.064019,0.182139,GasA,Gd,Y,SBrkr,0.326139,0.376477,0.0,0.548792,0.0,0.0,0.50,0.5,0.500000,0.5,TA,0.430677,Typ,0.25,TA,Attchd,0.678344,Fin,0.4,0.323925,TA,TA,Y,0.148876,0.045822,0.0,0.0,0.000000,0.0,Mising,MnPrv,Mising,0.000000,0.442114,0.0,WD,Normal,0.0,0.0,0.0,0.0,0.0
3,0.002058,0.487992,RL,0.582212,0.524583,Pave,Mising,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,0.778151,0.815465,0.100000,0.225806,Gable,CompShg,VinylSd,VinylSd,BrkFace,0.015504,TA,TA,PConc,TA,TA,No,GLQ,0.150125,Unf,0.000000,0.151402,0.181747,GasA,Ex,Y,SBrkr,0.325285,0.364125,0.0,0.542672,0.0,0.0,0.50,0.5,0.500000,0.5,Gd,0.526456,Typ,0.25,Gd,Attchd,0.675159,Fin,0.4,0.315860,TA,TA,Y,0.252809,0.048518,0.0,0.0,0.000000,0.0,Mising,Mising,Mising,0.000000,0.721057,0.0,WD,Normal,0.0,0.0,0.0,0.0,0.0
4,0.002743,0.795881,RL,0.317987,0.335596,Pave,Mising,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,0.903090,0.732487,0.146154,0.322581,Gable,CompShg,HdBoard,HdBoard,Mising,0.000000,Gd,TA,PConc,Gd,TA,No,ALQ,0.065586,Unf,0.000000,0.475234,0.251227,GasA,Ex,Y,SBrkr,0.453388,0.000000,0.0,0.453388,0.0,0.0,0.50,0.0,0.333333,0.5,Gd,0.317394,Typ,0.00,Mising,Attchd,0.694268,RFn,0.4,0.340054,TA,TA,Y,0.000000,0.110512,0.0,0.0,0.250000,0.0,Mising,Mising,Mising,0.000000,0.000000,0.0,WD,Normal,0.0,0.0,0.0,0.0,0.0
